In [1]:
from functools import cached_property
import re

TODAY = 'day10'
TEST_FILE_INPUT = f"./test_input_{TODAY}.txt"
TEST_FILE_INPUT2 = f"./test_input2_{TODAY}.txt"
FILE_INPUT = f"./input_{TODAY}.txt"


In [2]:
PATH_DICT = {
    # key = square symbol
    # value = dict
        # key = direction you came from
        # value = direction you will go
    # Example "L": {"north": "east", "east: "north"}
    
    "|": {"south": "north", "north": "south"},
    "-": {"west": "east", "east": "west"},
    "L": {"north": "east", "east": "north"},
    "J": {"west": "north", "north": "west"},
    "F": {"south": "east", "east": "south"},
    "7": {"west": "south", "south": "west"},
}

RELATIVE_POSITIONS_DICT = {
    "west": (0, -1),
    "east": (0, 1),
    "south": (1, 0),
    "north": (-1, 0)
}

OPPOSITE_DIRECTION_DICT = { # if you are to my <key> then I am to your <value>
    "north": "south",
    "south": "north",
    "east": "west",
    "west": "east",
}

class MapPosition:
    def __init__(self, grid_position, previous_location, map_dict):
        """
        Location on the map when traversing
        grid_position: where you are on the map (matrix coords)
        previous_location: direction (n/s/e/w) relative to the current location where you were previously
        map_dict: a dictionary showing where pipes are located
        """
        
        self.grid_position = grid_position
        self.map_dict = map_dict
        self.previous_location = previous_location
        self.path_positions = [self.grid_position]
        
    def move(self, direction):
        x, y = self.grid_position
        dx, dy = RELATIVE_POSITIONS_DICT[direction]
        new_position = (x+dx, y+dy)
        self.grid_position = new_position
        self.previous_location = OPPOSITE_DIRECTION_DICT[direction]
        self.path_positions.append(self.grid_position)
        
    def direction_to_move(self):
        current_pipe = self.map_dict[self.grid_position]
        possible_move_directions = list(PATH_DICT[current_pipe].keys())
        possible_move_directions.remove(self.previous_location)
        next_direction = possible_move_directions[0]
        return next_direction
        

class PipeMap:
    def __init__(self, file_path):
        self.file_path = file_path
        self.starting_position = None
        self.map_dict = None
        self.path_positions = None
        
    
    def process(self):
        valid_chars = PATH_DICT.keys()
        map_dict = {}
        with open(self.file_path, 'r') as f:
            for ii, line in enumerate(f):
                line = line.rstrip('\n')
                for jj, char in enumerate(line):
                    if char == "S": 
                        self.starting_position = (ii, jj)
                    if char in valid_chars:
                        map_dict[(ii, jj)] = char
        self.map_dict = map_dict
        
    def resolve_starting_position(self):
        if self.starting_position is None:
            self.process()
        if self.starting_position is None:
            print("No 'S' character found")
            return 
        
        connecting_pipe_positions = [] # neighbors of start (relative to start) that have connecting pipes
        print(f"S pipe is at position {self.starting_position}")
        for direction, diff in RELATIVE_POSITIONS_DICT.items():
            x, y = self.starting_position
            dx, dy = diff
            new_pos = (x + dx, y + dy)
            neighbor_pipe = self.map_dict.get(new_pos, None)
            if neighbor_pipe:
                print(f"   Neighbor pipe {neighbor_pipe} found to the {direction}")
            relative_direction = OPPOSITE_DIRECTION_DICT[direction]
            if neighbor_pipe and relative_direction in PATH_DICT[neighbor_pipe].keys():
                connecting_pipe_positions.append(direction)
                
        print(f"Connecting neighbor positions are to the {connecting_pipe_positions}")
        
        resolve_dict = {
            ("north", "south"): "|",
            ("south", "north"): "|",
            ("east", "west"): "-",
            ("west", "east"): "-",
            ("north", "west"): "J",
            ("west", "north"): "J",
            ("north", "east"): "L",
            ("east", "north"): "L",
            ("south", "east"): "F",
            ("east", "south"): "F",
            ("south", "west"): "7",
            ("west", "south"): "7",
        }
        
        starting_char = resolve_dict[tuple(connecting_pipe_positions)]
        print(f"...so the starting pipe is {starting_char}")
                
        self.map_dict[self.starting_position] = starting_char
            
    def traverse(self, starting_position = None):
        if starting_position is None:
            starting_position = self.starting_position
        print(f"Traversing path starting from {starting_position}")
        pipe_type = self.map_dict[starting_position]
        previous_direction = list(PATH_DICT[pipe_type].keys())[0] # arbitrary
        position = MapPosition(
            grid_position = starting_position,
            previous_location = previous_direction,
            map_dict = self.map_dict
        )
        position.move(direction = position.direction_to_move())
        path_length = 1
        
        while position.grid_position != starting_position:
            # print(f"Currently at position {position.grid_position}")
            position.move(direction = position.direction_to_move())
            
            path_length += 1
            
        print(f"Returned to {starting_position} after {path_length} steps")
        
        self.path_positions = position.path_positions
        
        return path_length // 2
        

class PartA:
    def __init__(self, pipe_map):
        self.pipe_map = pipe_map        

    def solve(self):
        self.pipe_map.process()
        self.pipe_map.resolve_starting_position()
        return self.pipe_map.traverse() 

In [3]:
pipe_map_a_test = PipeMap(TEST_FILE_INPUT)

In [4]:
a_test = PartA(pipe_map_a_test)
assert a_test.solve() == 4

S pipe is at position (1, 1)
   Neighbor pipe - found to the east
   Neighbor pipe | found to the south
Connecting neighbor positions are to the ['east', 'south']
...so the starting pipe is F
Traversing path starting from (1, 1)
Returned to (1, 1) after 8 steps


In [5]:
a_test.pipe_map.path_positions

[(1, 1), (1, 2), (1, 3), (2, 3), (3, 3), (3, 2), (3, 1), (2, 1), (1, 1)]

In [6]:
pipe_map_a_test2 = PipeMap(TEST_FILE_INPUT2)
a_test2 = PartA(pipe_map_a_test2)
assert a_test2.solve() == 8

S pipe is at position (2, 0)
   Neighbor pipe J found to the east
   Neighbor pipe | found to the south
Connecting neighbor positions are to the ['east', 'south']
...so the starting pipe is F
Traversing path starting from (2, 0)
Returned to (2, 0) after 16 steps


In [7]:
pipe_map_a = PipeMap(FILE_INPUT)
part_a = PartA(pipe_map_a)
part_a.solve()

S pipe is at position (38, 55)
   Neighbor pipe 7 found to the west
   Neighbor pipe L found to the east
   Neighbor pipe | found to the south
   Neighbor pipe | found to the north
Connecting neighbor positions are to the ['south', 'north']
...so the starting pipe is |
Traversing path starting from (38, 55)
Returned to (38, 55) after 14172 steps


7086

In [ ]:
class PartB(PartA):    
    def solve(self):
        pass
                    
            
        

In [ ]:
pipe_map_b_test = PipeMap(TEST_FILE_INPUT)

In [ ]:
b_test = PartB(pipe_map_b_test)
assert b_test.solve() == 4

In [ ]:
b = PartB(FILE_INPUT)
b.solve()